In [ ]:
pip install psycopg2

In [ ]:
import os
import pandas as pd
import psycopg2

# Set folder path and DB info
folder_path = r"C:\Users\HP\Desktop\olist_data"
db_name = "sales_db"
user = "postgres"  # replace if yours is different
password = "admin123"  # 👈 replace this with your actual password
host = "localhost"
port = "5432"

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname=db_name, user=user, password=password, host=host, port=port
)
cur = conn.cursor()

# Mapping pandas dtypes to PostgreSQL types
def map_dtype(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(dtype):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "TIMESTAMP"
    else:
        return "TEXT"

# Loop through all CSV files
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        table_name = os.path.splitext(filename)[0].lower()

        # Read CSV
        df = pd.read_csv(file_path)

        # Clean column names
        df.columns = [col.strip().lower().replace(" ", "_").replace("-", "_") for col in df.columns]

        # Create SQL table
        columns = [
            f'"{col}" {map_dtype(df[col])}' for col in df.columns
        ]
        create_table_sql = f'CREATE TABLE IF NOT EXISTS "{table_name}" (\n  {", ".join(columns)}\n);'
        cur.execute(create_table_sql)
        conn.commit()

        # Insert data
        for _, row in df.iterrows():
            values = [None if pd.isna(val) else val for val in row]
            placeholders = ", ".join(["%s"] * len(values))
            insert_sql = f'INSERT INTO "{table_name}" VALUES ({placeholders});'
            cur.execute(insert_sql, values)

        conn.commit()
        print(f"✅ Loaded: {filename} into table '{table_name}'")

# Close connection
cur.close()
conn.close()
print("🎉 All files loaded successfully into sales_db!")

